## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,53.24,83,0,3.44,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,38.86,81,75,16.11,broken clouds
2,2,Asind,IN,25.7333,74.3333,87.06,28,0,16.58,clear sky
3,3,Mehtar Lam,AF,34.6714,70.2094,78.30,19,13,4.70,few clouds
4,4,Vaini,TO,-21.2000,-175.2000,78.96,61,20,14.97,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 40
What is the maximum temperature you would like for your vacation? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,53.24,83,0,3.44,clear sky
2,2,Asind,IN,25.7333,74.3333,87.06,28,0,16.58,clear sky
3,3,Mehtar Lam,AF,34.6714,70.2094,78.30,19,13,4.70,few clouds
4,4,Vaini,TO,-21.2000,-175.2000,78.96,61,20,14.97,few clouds
5,5,Souillac,MU,-20.5167,57.5167,73.78,88,20,9.22,few clouds
6,6,Sembe,CG,1.6481,14.5806,68.68,98,100,2.01,overcast clouds
9,9,Gusau,NG,12.1628,6.6614,84.92,17,86,7.11,overcast clouds
10,10,Jamestown,US,42.0970,-79.2353,42.39,60,100,17.27,overcast clouds
12,12,Buin,CL,-33.7333,-70.7500,51.84,82,100,4.83,overcast clouds
15,15,Necochea,AR,-38.5473,-58.7368,62.76,99,100,7.47,heavy intensity rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,53.24,83,0,3.44,clear sky
2,2,Asind,IN,25.7333,74.3333,87.06,28,0,16.58,clear sky
3,3,Mehtar Lam,AF,34.6714,70.2094,78.30,19,13,4.70,few clouds
4,4,Vaini,TO,-21.2000,-175.2000,78.96,61,20,14.97,few clouds
5,5,Souillac,MU,-20.5167,57.5167,73.78,88,20,9.22,few clouds


In [7]:
clean_travel_cities.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bredasdorp,ZA,53.24,clear sky,-34.5322,20.0403,
2,Asind,IN,87.06,clear sky,25.7333,74.3333,
3,Mehtar Lam,AF,78.30,few clouds,34.6714,70.2094,
4,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
5,Souillac,MU,73.78,few clouds,-20.5167,57.5167,
6,Sembe,CG,68.68,overcast clouds,1.6481,14.5806,
9,Gusau,NG,84.92,overcast clouds,12.1628,6.6614,
10,Jamestown,US,42.39,overcast clouds,42.0970,-79.2353,
12,Buin,CL,51.84,overcast clouds,-33.7333,-70.7500,
15,Necochea,AR,62.76,heavy intensity rain,-38.5473,-58.7368,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bredasdorp,ZA,53.24,clear sky,-34.5322,20.0403,Bredasdorp Country Manor
2,Asind,IN,87.06,clear sky,25.7333,74.3333,"HOTEL JIYA PALACE, ASIND"
3,Mehtar Lam,AF,78.30,few clouds,34.6714,70.2094,Fareed Hakamani
4,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Souillac,MU,73.78,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
9,Gusau,NG,84.92,overcast clouds,12.1628,6.6614,GUEST INN
10,Jamestown,US,42.39,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
12,Buin,CL,51.84,overcast clouds,-33.7333,-70.7500,Paihuen
15,Necochea,AR,62.76,heavy intensity rain,-38.5473,-58.7368,dyd -mar Hotel
16,Rikitea,PF,77.88,overcast clouds,-23.1203,-134.9692,People ThankYou


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))